In [1]:
import SimpleITK as sitk
resample = sitk.ResampleImageFilter()
# resample.SetInterpolator(sitk.sitkNearest)
dir(sitk)

['Abs',
 'AbsImageFilter',
 'AbsoluteValueDifference',
 'AbsoluteValueDifferenceImageFilter',
 'Acos',
 'AcosImageFilter',
 'AdaptiveHistogramEqualization',
 'AdaptiveHistogramEqualizationImageFilter',
 'Add',
 'AddImageFilter',
 'AdditiveGaussianNoise',
 'AdditiveGaussianNoiseImageFilter',
 'AffineTransform',
 'AggregateLabelMap',
 'AggregateLabelMapFilter',
 'And',
 'AndImageFilter',
 'AntiAliasBinary',
 'AntiAliasBinaryImageFilter',
 'ApproximateSignedDistanceMap',
 'ApproximateSignedDistanceMapImageFilter',
 'Asin',
 'AsinImageFilter',
 'Atan',
 'Atan2',
 'Atan2ImageFilter',
 'AtanImageFilter',
 'BSplineDecomposition',
 'BSplineDecompositionImageFilter',
 'BSplineTransform',
 'BSplineTransformInitializer',
 'BSplineTransformInitializerFilter',
 'Bilateral',
 'BilateralImageFilter',
 'BinShrink',
 'BinShrinkImageFilter',
 'BinaryClosingByReconstruction',
 'BinaryClosingByReconstructionImageFilter',
 'BinaryContour',
 'BinaryContourImageFilter',
 'BinaryDilate',
 'BinaryDilateImageFi

In [2]:
import numpy as np
from glob import glob
from tqdm import tqdm
import h5py
import nrrd
import pandas as pd
import SimpleITK as sitk

output_size =[128, 128, 64]

def resample_image3D(
    image3D,
    newspacing=[0.3,0.3,3],
    newsize=None,
    method='Linear',):
    """做插值"""
    resample = sitk.ResampleImageFilter()
    if method == 'Linear':
        resample.SetInterpolator(sitk.sitkLinear)
    elif method == 'Nearest':
        resample.SetInterpolator(sitk.sitkNearestNeighbor)
    resample.SetOutputDirection(image3D.GetDirection())
    resample.SetOutputOrigin(image3D.GetOrigin())
    resample.SetOutputSpacing(newspacing)

    if not newsize:
        newsize = np.round(np.array(image3D.GetSize())*np.abs(image3D.GetSpacing())/np.array(newspacing)).astype('int').tolist()

    resample.SetSize(newsize)
    # resample.SetDefaultPixelValue(0)

    newimage = resample.Execute(image3D)
#     data1 = sitk.GetArrayFromImage(newimage)
#     print(data1)
    return newimage

# 数组替换元素
def array_replace(array,olds,news):
    #olds:list of old value
    #news:list of new value
    olds = np.array(olds)
    news = np.array(news)
    offset = olds.max()*10
    tmps = olds+offset
    for old,tmp in zip(olds,tmps):
        array[array==old] = tmp
    for tmp,new in zip(tmps,news):
        array[array==tmp] = new
    return array
    
    
def get_stats(glob_str, old_replaced, new_replaced):
    """
    获取数据集的统计信息。
    备注：不要骨头，骨头合并到背景类别中
    """
    listt = glob(glob_str)
    error_samples = []
    stats = pd.DataFrame(columns=['sample_name',
                                  'mean_whole', 
                                  'mean_bg', 
                                  'mean_dura', 
                                  'mean_SC', 
                                  'std_whole',
                                  'std_bg',
                                  'std_dura',
                                  'std_SC',
                                  'old_space0','old_space1','old_space2',
                                  'new_space0','new_space1','new_space2',
                                 ])
    for item in tqdm(listt):
        sample_name = item.split('/')[-2]
        print(sample_name,':')#win系统改为'\\'

        image = sitk.ReadImage(item)
        label = sitk.ReadImage(item.replace(old_replaced, 'Segmentation-label.nrrd'))
        seg = sitk.ReadImage(item.replace(old_replaced, 'Segmentation.seg.nrrd'))
        
        oldspacing = np.abs(image.GetSpacing())
        newspacing = [0.3, 0.3, 3.0]
        offset = [int(k) for k in seg.GetMetaData('Segmentation_ReferenceImageExtentOffset').split()]
        print("Offset:",offset)
        offset = ( np.array(offset)*np.array(oldspacing)/np.array(newspacing) ).astype(int).tolist()
        label_name = [
            seg.GetMetaData('Segment0_Name'),
            seg.GetMetaData('Segment1_Name'),
            seg.GetMetaData('Segment2_Name') 
            ]#人工标注的类别顺序
        
        # resample
        image = resample_image3D(image,newspacing,method='Linear')
        label = resample_image3D(label,newspacing,method='Nearest')#,newsize=image.GetSize()
        seg = resample_image3D(seg,newspacing)
        
        # cut
        sizes = seg.GetSize()#resample后seg的size
        image = sitk.GetArrayFromImage(image).transpose((2,1,0))#tanspose之后才能与sizes匹配
        label = sitk.GetArrayFromImage(label).transpose((2,1,0))#tanspose之后才能与sizes匹配
        seg = sitk.GetArrayFromImage(seg).transpose((3,2,1,0))#tanspose之后才能与sizes匹配
        
        if not image.shape==label.shape:
#             import pdb
#             pdb.set_trace()
            print('stop')
        assert np.abs(np.array(image.shape)-np.array(label.shape)).sum()<=1, "mismatch shape of image and label"
            
            
        image = image[offset[0]:offset[0]+sizes[0],
                      offset[1]:offset[1]+sizes[1],
                      offset[2]:offset[2]+sizes[2]]
        label = label[offset[0]:offset[0]+sizes[0],
                      offset[1]:offset[1]+sizes[1],
                      offset[2]:offset[2]+sizes[2]].astype(np.uint8)
        
#         import pdb
#         pdb.set_trace()
        print(image.shape)
        # 错误病例：标记的尺寸和image尺寸不同，缺少其中一个类别或者多个类别的标记
        if not np.unique(label).tolist()==[0,1,2,3]:
            error_samples.append(sample_name)
            print("error sample(no dura/SC):",sample_name)
            continue
        
        if not np.abs(np.array(image.shape)-np.array(seg.shape[1::])).sum()<=1:
            error_samples.append(sample_name)
            print("error sample(shape mismatch):",sample_name)
            continue
            
        
        ## 调整顺序，注意：seg是onehot编码
        target_name = ['dura','bone','SC']#目标类别顺序
        idx = [label_name.index(name) for name in target_name]
        idx = np.array(idx)+1
        idx = [0]+idx.tolist()
        label = array_replace(label,olds=[0,1,2,3],news=idx)
        
        # 合并骨头到背景中
        label[label==2] = 0
        label[label==3] = 2
        
        # 统计信息
        
        # std
        std_whole = np.std(image)
        std_bg = np.std(image[label==0])
        std_dura = np.std(image[label==1])
        std_SC = np.std(image[label==2])
        # mean
        mean_whole = np.mean(image)
        mean_bg = np.mean(image[label==0])
        mean_dura = np.mean(image[label==1])
        mean_SC = np.mean(image[label==2])
        
        stats.loc[sample_name]=[sample_name,
                                mean_whole,mean_bg,mean_dura,mean_SC,
                                std_whole,std_bg,std_dura,std_SC,
                                oldspacing[0],oldspacing[1],oldspacing[2],
                                newspacing[0],newspacing[1],newspacing[2],
                               ]   
        
    return error_samples,stats

def covert_h5_unseg(glob_str, old_replaced, new_replaced):
    """
    备注：无标注数据的格式转换
    """
    listt = glob(glob_str)
    for item in tqdm(listt):
        print(item.split('/')[-2],':')
        # 读取原始图
        image, img_header = nrrd.read(item) 
        
#         # 缩小图像
#         image = image[0:-1:2,0:-1:2,:]

#         # 标准化
#         image = (image - np.mean(image)) / np.std(image)
#         image = image.astype(np.float32)
#         print(image.shape)
        
        f = h5py.File(item.replace(old_replaced, new_replaced), 'w')
        f.create_dataset('image', data=image, compression="gzip")
        f.close()  
    print("total number of unseg-samples:", len(listt))

    
if __name__ == '__main__':
#     # 有标签数据
#     print('seg dataset:')
#     glob_str = '../../data/CTM_dataset/Segmented/*/CTM.nrrd'
#     error_samples,stats = get_stats(glob_str,'CTM.nrrd','mri_norm2.h5')
#     display(stats)
#     print("error_samples:",error_samples)
#     stats.to_csv('../../data/CTM_dataset/Segmented/stats.csv')
    # 无标签数据
    print('unseg dataset:')
    glob_str = '../../data/CTM_dataset/unSegmented/*/CTM.nrrd'
    covert_h5_unseg(glob_str,'CTM.nrrd','mri_norm2.h5')  
    


  0%|          | 0/797 [00:00<?, ?it/s]

unseg dataset:
1204979 :


  0%|          | 1/797 [00:01<20:40,  1.56s/it]

b1261721 :


  0%|          | 2/797 [00:03<20:59,  1.58s/it]

...

 96%|█████████▌| 763/797 [23:03<01:07,  1.99s/it]

b1539233 :


 96%|█████████▌| 764/797 [23:05<00:59,  1.81s/it]

1187634 :


error: Error -3 while decompressing data: invalid literal/length code